#  使用重复元素的网络（VGG）

VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为3 \* 3的卷积层后,接上一个步幅为2 、窗口形状为2 * 2 的最大池化层。

In [1]:
import sys
import time
sys.path.append('..')

In [2]:
import mxnet as mx
from mxnet import gluon,nd,init
from mxnet.gluon import nn,data as gdata,loss as gloss

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import gluonbook as gb

In [4]:
def vgg_block(num_convs,num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels,kernel_size=3,padding=1,activation='relu')
        )
    #卷积层之后加入
    blk.add(nn.MaxPool2D(pool_size=2,strides=2))
    return blk

## 通AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个vgg_block，其超参数由变量 conv_arch 定义。该变量指定了每个 VGG 块⾥卷积层个数和输出通道数。全连接模块则跟 AlexNet 中的⼀样

现在我们构造⼀个 VGG ⽹络。它有 5 个卷积块，前两块使⽤单卷积层，而后三块使⽤双卷积层。
第⼀块的输出通道是 64，之后每次对输出通道数翻倍，直到变为 512。因为这个⽹络使⽤了 8 个
卷积层和 3 个全连接层，所以经常被称为 VGG-11。

In [5]:
conv_arch = ((1,64),(1,128),(2,256),(2,512),(2,512))

In [6]:
def vgg(conv_arch):
    VGG = nn.Sequential()
    #卷积层部分
    for num_convs,num_channels in conv_arch:
        VGG.add(vgg_block(num_convs,num_channels))
    #全连接层部分
    VGG.add(nn.Dense(128,activation='relu'),nn.Dropout(0.5),
            nn.Dense(128,activation='relu'),nn.Dropout(0.5),
            nn.Dense(10))
    return VGG

In [7]:
VGG = vgg(conv_arch)

In [8]:
VGG.initialize(ctx=mx.gpu())

X =nd.random.uniform(shape=(1,1,224,224),ctx=mx.gpu())

for layer in VGG:
    X = layer(X)
    print(layer.name,'output shape',X.shape)

sequential1 output shape (1, 64, 112, 112)
sequential2 output shape (1, 128, 56, 56)
sequential3 output shape (1, 256, 28, 28)
sequential4 output shape (1, 512, 14, 14)
sequential5 output shape (1, 512, 7, 7)
dense0 output shape (1, 128)
dropout0 output shape (1, 128)
dense1 output shape (1, 128)
dropout1 output shape (1, 128)
dense2 output shape (1, 10)


In [9]:
ratio = 16

small_conv_arch = [(pair[0],pair[1] // ratio) for pair in conv_arch]
small_VGG = vgg(small_conv_arch)

In [10]:
lr, num_epochs, batch_size, ctx = 0.05, 5, 128, gb.try_gpu()
small_VGG.initialize(ctx=ctx, init=init.Xavier(),force_reinit=True)
trainer = gluon.Trainer(small_VGG.collect_params(), 'sgd', {'learning_rate': lr})

def load_data_fashion_mnist(batch_size,resize=None):
    transformer = []
    path = '../chapter1_baseKnowledge/FashionMNIST/'
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    mnist_train =gdata.vision.FashionMNIST(root=path,train=True)
    mnist_test =gdata.vision.FashionMNIST(root= path,train=False)
    
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),batch_size,shuffle = True)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),batch_size,shuffle=False)
    
    return train_iter,test_iter

In [11]:
train_iter,test_iter = load_data_fashion_mnist(batch_size =batch_size,resize=224)

gb.train_ch5(small_VGG, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 1.1843, train acc 0.545, test acc 0.785, time 92.5 sec
epoch 2, loss 0.6306, train acc 0.767, test acc 0.842, time 88.1 sec
epoch 3, loss 0.5161, train acc 0.817, test acc 0.871, time 88.2 sec
epoch 4, loss 0.4532, train acc 0.840, test acc 0.876, time 88.0 sec
epoch 5, loss 0.4142, train acc 0.856, test acc 0.876, time 88.1 sec
